In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

2025-07-14 15:55:43.111809: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-14 15:55:43.252385: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-14 15:55:43.290607: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-14 15:55:43.994777: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:

# Загрузка и агрегация данных
data = pd.read_csv("filtered_dataset.csv", parse_dates=["date"])
data_daily = data.set_index('date').resample('D').mean().reset_index()
data_daily['value'] = data_daily['value'].interpolate(method='linear')



In [3]:
# Нормализация
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_daily[['value']])

In [4]:
# Формирование обучающих последовательностей
sequence_length = 7
X, y = [], []
for i in range(len(data_scaled) - sequence_length):
    X.append(data_scaled[i:i+sequence_length])
    y.append(data_scaled[i+sequence_length])
X, y = np.array(X), np.array(y)

In [5]:
# Разделение на train/test
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
# Модель GRU
model = Sequential([
    GRU(50, input_shape=(sequence_length, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

2025-07-14 15:55:44.760492: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-14 15:55:44.768381: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-14 15:55:44.768811: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-14 15:55:44.769984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/20


2025-07-14 15:55:47.633659: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


 1/21 [>.............................] - ETA: 57s - loss: 0.2219

2025-07-14 15:55:48.567188: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


21/21 [==============================] - 3s 22ms/step - loss: 0.0508 - val_loss: 0.0063
Epoch 2/20
21/21 [==============================] - 0s 8ms/step - loss: 0.0074 - val_loss: 0.0154
Epoch 3/20
21/21 [==============================] - 0s 7ms/step - loss: 0.0054 - val_loss: 0.0085
Epoch 4/20
21/21 [==============================] - 0s 9ms/step - loss: 0.0045 - val_loss: 0.0064
Epoch 5/20
21/21 [==============================] - 0s 8ms/step - loss: 0.0041 - val_loss: 0.0047
Epoch 6/20
21/21 [==============================] - 0s 9ms/step - loss: 0.0040 - val_loss: 0.0048
Epoch 7/20
21/21 [==============================] - 0s 9ms/step - loss: 0.0040 - val_loss: 0.0042
Epoch 8/20
21/21 [==============================] - 0s 9ms/step - loss: 0.0040 - val_loss: 0.0050
Epoch 9/20
21/21 [==============================] - 0s 9ms/step - loss: 0.0039 - val_loss: 0.0051
Epoch 10/20
21/21 [==============================] - 0s 9ms/step - loss: 0.0039 - val_loss: 0.0050
Epoch 11/20
21/21 [==========

In [8]:
# Прогноз на 2 дня вперёд
last_sequence = data_scaled[-sequence_length:]
for _ in range(10):
    pred = model.predict(last_sequence.reshape(1, sequence_length, 1))
    print('Прогноз:', scaler.inverse_transform(pred))
    last_sequence = np.append(last_sequence[1:], pred).reshape(sequence_length, 1)


1/1 [==============================] - 0s 281ms/step
Прогноз: [[1.9735982]]
1/1 [==============================] - 0s 24ms/step
Прогноз: [[1.966381]]
1/1 [==============================] - 0s 22ms/step
Прогноз: [[1.9607514]]
1/1 [==============================] - 0s 23ms/step
Прогноз: [[1.9560223]]
1/1 [==============================] - 0s 21ms/step
Прогноз: [[1.9516114]]
1/1 [==============================] - 0s 21ms/step
Прогноз: [[1.9470956]]
1/1 [==============================] - 0s 21ms/step
Прогноз: [[1.9425513]]
1/1 [==============================] - 0s 22ms/step
Прогноз: [[1.9383104]]
1/1 [==============================] - 0s 21ms/step
Прогноз: [[1.9340909]]
1/1 [==============================] - 0s 22ms/step
Прогноз: [[1.9298649]]
